In [3]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score

In [8]:
# Load the dataset
df = pd.read_csv("delhi_routes.csv")

def preprocess_data(df):
    # Drop columns with missing congestion and waterlogging points
    df_cleaned = df.drop(columns=["Congestion Points", "Waterlogging Points"])
    
    # Encode categorical columns
    label_encoders = {}
    for col in ["Start", "Destination", "Via Road", "Route Type"]:
        le = LabelEncoder()
        df_cleaned[col] = le.fit_transform(df_cleaned[col])
        label_encoders[col] = le
    joblib.dump(label_encoders, "label_encoders.pkl")
    
    # Normalize numerical columns
    scaler = MinMaxScaler()
    scaled_features = scaler.fit_transform(df_cleaned.iloc[:, 2:])
    df_scaled = pd.DataFrame(scaled_features, columns=df_cleaned.columns[2:])
    joblib.dump(scaler, "scaler.pkl")
    
    return df_scaled

# Preprocess data
df_scaled = preprocess_data(df)
X = df_scaled.drop(columns=["Total Time", "Min Time"])
y = df_scaled[["Total Time", "Min Time"]]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(2, activation='linear')
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss = 'mean_squared_error', metrics=['mae'])

# Train model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Save model
model.save("ann_route_prediction.h5")

# Evaluate model
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2:.4f}")


Epoch 1/100


C:\Users\adars\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1008 - mae: 0.2318 - val_loss: 0.0024 - val_mae: 0.0385
Epoch 2/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0171 - mae: 0.1011 - val_loss: 0.0032 - val_mae: 0.0471
Epoch 3/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0123 - mae: 0.0848 - val_loss: 0.0017 - val_mae: 0.0328
Epoch 4/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0101 - mae: 0.0773 - val_loss: 0.0024 - val_mae: 0.0385
Epoch 5/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0082 - mae: 0.0694 - val_loss: 0.0013 - val_mae: 0.0295
Epoch 6/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0071 - mae: 0.0653 - val_loss: 0.0023 - val_mae: 0.0394
Epoch 7/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0066 - mae: 0.0624 - val_loss: 0.0021 - val_mae: 0.0372
Epoch 8/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0058 - mae: 0.0584 - val_loss: 0.0012 - val_mae: 0.0277
Epoch 9/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss:

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
R² Score: 0.9938
